In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers.reshaping.flatten import Flatten
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras.preprocessing.image import ImageDataGenerator 

In [6]:
train_data_gen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
validation_data_gen = ImageDataGenerator(rescale=1./255,
                                        rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')

In [7]:
train_generator=train_data_gen.flow_from_directory(
    'images/train',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical')

Found 28821 images belonging to 7 classes.


In [26]:
validation_generator=validation_data_gen.flow_from_directory(
    'images/validation',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical')

Found 7066 images belonging to 7 classes.


In [27]:
from keras.optimizers import Adam

In [28]:
model = Sequential()

model.add(Conv2D(32,kernel_size = (3,3),padding='valid',activation='relu',input_shape=(48,48,1)))
model.add(Conv2D(64,kernel_size = (3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='valid'))
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size = (3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,kernel_size = (3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='valid'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001,decay=1e-6),metrics=['Accuracy'])

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 conv2d_13 (Conv2D)          (None, 44, 44, 64)        18496     
                                                                 
 batch_normalization_6 (Batc  (None, 44, 44, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 22, 22, 64)       0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 22, 22, 64)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 20, 20, 128)      

In [31]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=28821//64,
                              epochs=50,
                              validation_data=validation_generator,
                              validation_steps=110)

C:\Users\vikas\AppData\Local\Temp\ipykernel_27960\576839271.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/50
450/450 [==============================] - 1330s 3s/step - loss: 2.1506 - Accuracy: 0.2092 - val_loss: 2.2300 - val_Accuracy: 0.1807
Epoch 2/50
450/450 [==============================] - 439s 974ms/step - loss: 1.8396 - Accuracy: 0.2319 - val_loss: 1.7952 - val_Accuracy: 0.2626
Epoch 3/50
450/450 [==============================] - 335s 744ms/step - loss: 1.8070 - Accuracy: 0.2453 - val_loss: 1.8061 - val_Accuracy: 0.2592
Epoch 4/50
450/450 [==============================] - 418s 930ms/step - loss: 1.7916 - Accuracy: 0.2577 - val_loss: 1.8170 - val_Accuracy: 0.2690
Epoch 5/50
450/450 [==============================] - 332s 737ms/step - loss: 1.7819 - Accuracy: 0.2599 - val_loss: 1.7504 - val_Accuracy: 0.2916
Epoch 6/50
450/450 [==============================] - 331s 735ms/step - loss: 1.7733 - Accuracy: 0.2683 - val_loss: 2.0131 - val_Accuracy: 0.2714
Epoch 7/50
450/450 [==============================] - 334s 741ms/step - loss: 1.7574 - Accuracy: 0.2793 - val_loss: 1.7716 - v

In [33]:
print('hello world')

hello world


In [35]:
model_json=model.to_json()# Storing the model structure in a json file
with open("fer_model.json","w") as json_file:
    json_file.write(model_json)

In [36]:
model.save_weights('final_model.h5')